In [1]:
nombre = "\n Ricardo Madrigal Urencio \n"
print(nombre)


 Ricardo Madrigal Urencio 



In [2]:
#Todas las librerias a Importar
from pyspark.sql.functions import col, sum, to_date, to_timestamp, regexp_replace
from pyspark.sql.functions import date_format
from pyspark.sql.types import IntegerType, DoubleType, FloatType
from pyspark.sql.functions import hour, minute, second, year, month, dayofmonth, weekofyear

In [3]:
#Crear un solo Dataframe con todos los meses
dir_hdfs = "hdfs://namenode:9000/tmp/amd/ecobici/2019"

df = spark.read.csv(dir_hdfs, header=True, inferSchema=True)
df.printSchema()

print(nombre)

root
 |-- Genero_Usuario: string (nullable = true)
 |-- Edad_Usuario: string (nullable = true)
 |-- Bici: string (nullable = true)
 |-- Ciclo_Estacion_Retiro: string (nullable = true)
 |-- Fecha_Retiro: string (nullable = true)
 |-- Hora_Retiro: string (nullable = true)
 |-- Ciclo_Estacion_Arribo: string (nullable = true)
 |-- Fecha_Arribo: string (nullable = true)
 |-- Hora_Arribo: string (nullable = true)
 |-- _c9: timestamp (nullable = true)


 Ricardo Madrigal Urencio 



In [4]:
print(df.count())
print(nombre)

7718156

 Ricardo Madrigal Urencio 



In [5]:
df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()
print(nombre)

+--------------+------------+----+---------------------+------------+-----------+---------------------+------------+-----------+-------+
|Genero_Usuario|Edad_Usuario|Bici|Ciclo_Estacion_Retiro|Fecha_Retiro|Hora_Retiro|Ciclo_Estacion_Arribo|Fecha_Arribo|Hora_Arribo|    _c9|
+--------------+------------+----+---------------------+------------+-----------+---------------------+------------+-----------+-------+
|             0|           0|   0|                    0|           0|          0|                    0|           0|          0|7718155|
+--------------+------------+----+---------------------+------------+-----------+---------------------+------------+-----------+-------+


 Ricardo Madrigal Urencio 



In [6]:
df.show(10, truncate = False)

print(nombre)

+--------------+------------+-----+---------------------+------------+-----------+---------------------+------------+-----------+----+
|Genero_Usuario|Edad_Usuario|Bici |Ciclo_Estacion_Retiro|Fecha_Retiro|Hora_Retiro|Ciclo_Estacion_Arribo|Fecha_Arribo|Hora_Arribo|_c9 |
+--------------+------------+-----+---------------------+------------+-----------+---------------------+------------+-----------+----+
|M             |34          |8861 |143                  |01/03/2019  |00:00:54   |68                   |01/03/2019  |00:13:55   |NULL|
|F             |33          |2601 |51                   |01/03/2019  |00:01:01   |53                   |01/03/2019  |00:07:16   |NULL|
|M             |25          |8257 |340                  |01/03/2019  |00:01:26   |324                  |01/03/2019  |00:09:26   |NULL|
|M             |32          |15007|139                  |01/03/2019  |00:01:42   |155                  |01/03/2019  |00:06:20   |NULL|
|F             |22          |7601 |81                  

In [7]:
df.filter("_c9 is not null").show(truncate = False)

print(nombre)

+--------------+------------+----+---------------------+------------+-----------+---------------------+------------+-----------+-------------------+
|Genero_Usuario|Edad_Usuario|Bici|Ciclo_Estacion_Retiro|Fecha_Retiro|Hora_Retiro|Ciclo_Estacion_Arribo|Fecha_Arribo|Hora_Arribo|_c9                |
+--------------+------------+----+---------------------+------------+-----------+---------------------+------------+-----------+-------------------+
|M             |44          |9944|267                  |06/03/2019  |08:06:23   |32:11.8              |10          |00:00:00   |2024-10-20 08:41:11|
+--------------+------------+----+---------------------+------------+-----------+---------------------+------------+-----------+-------------------+


 Ricardo Madrigal Urencio 



In [8]:
#Limpieza del esquema
datos = df\
.withColumn("Edad_Usuario", col("Edad_Usuario").cast(FloatType())) \
.withColumn("Bici", col("Bici").cast(IntegerType())) \
.withColumn("Ciclo_Estacion_Retiro", col("Ciclo_Estacion_Retiro").cast(IntegerType())) \
.withColumn("Fecha_Retiro", to_date(col("Fecha_Retiro"), "dd/MM/yyyy")) \
.withColumn("Hora_Retiro", date_format(col("Hora_Retiro"), "HH:mm:ss")) \
.withColumn("Ciclo_Estacion_Arribo", col("Ciclo_Estacion_Arribo").cast(IntegerType())) \
.withColumn("Fecha_Arribo", to_date(col("Fecha_Arribo"), "dd/MM/yyyy")) \
.withColumn("Hora_Arribo", date_format(col("Hora_Arribo"), "HH:mm:ss"))
datos.printSchema()

print(nombre)

root
 |-- Genero_Usuario: string (nullable = true)
 |-- Edad_Usuario: float (nullable = true)
 |-- Bici: integer (nullable = true)
 |-- Ciclo_Estacion_Retiro: integer (nullable = true)
 |-- Fecha_Retiro: date (nullable = true)
 |-- Hora_Retiro: string (nullable = true)
 |-- Ciclo_Estacion_Arribo: integer (nullable = true)
 |-- Fecha_Arribo: date (nullable = true)
 |-- Hora_Arribo: string (nullable = true)
 |-- _c9: timestamp (nullable = true)


 Ricardo Madrigal Urencio 



In [9]:
#Agregar campos año, mes, día, hora, minuto, segundo, día de la semana
##Separar Fechas y Horas de Retiro y Arribo en Campos Individuales
colfs = "Fecha_Retiro"
colhs = "Hora_Retiro"
colfa = "Fecha_Arribo"
colha = "Hora_Arribo"

ecobici = datos.select(
    col("Genero_Usuario"), col("Edad_Usuario"), col("Bici"), col("Ciclo_Estacion_Retiro"),
    date_format(col(colfs), "EEEE").alias("dia_sem_retiro"),
    dayofmonth(col(colfs)).alias("num_dia_retiro"),
    month(col(colfs)).alias("num_mes_retiro"),
    date_format(col(colfs), "MMMM").alias("mes_retiro"),
    year(col(colfs)).alias("anio_retiro"),
    hour(col(colhs)).alias("hora_retiro"),
    minute(col(colhs)).alias("minuto_retiro"),    
    second(col(colhs)).alias("segundo_retiro"),
    col("Ciclo_Estacion_Arribo"),
    date_format(col(colfa), "EEEE").alias("dia_sem_arribo"),
    dayofmonth(col(colfa)).alias("num_dia_arribo"),
    month(col(colfa)).alias("num_mes_arribo"),
    date_format(col(colfa), "MMMM").alias("mes_arribo"),
    year(col(colfa)).alias("anio_arribo"),
    hour(col(colha)).alias("hora_arribo"),
    minute(col(colha)).alias("minuto_arribo"),    
    second(col(colha)).alias("segundo_arribo")
    
)

ecobici.printSchema()

print(nombre)

ecobici.orderBy('num_mes_retiro', ascending = True).show(5)

print(nombre)

root
 |-- Genero_Usuario: string (nullable = true)
 |-- Edad_Usuario: float (nullable = true)
 |-- Bici: integer (nullable = true)
 |-- Ciclo_Estacion_Retiro: integer (nullable = true)
 |-- dia_sem_retiro: string (nullable = true)
 |-- num_dia_retiro: integer (nullable = true)
 |-- num_mes_retiro: integer (nullable = true)
 |-- mes_retiro: string (nullable = true)
 |-- anio_retiro: integer (nullable = true)
 |-- hora_retiro: integer (nullable = true)
 |-- minuto_retiro: integer (nullable = true)
 |-- segundo_retiro: integer (nullable = true)
 |-- Ciclo_Estacion_Arribo: integer (nullable = true)
 |-- dia_sem_arribo: string (nullable = true)
 |-- num_dia_arribo: integer (nullable = true)
 |-- num_mes_arribo: integer (nullable = true)
 |-- mes_arribo: string (nullable = true)
 |-- anio_arribo: integer (nullable = true)
 |-- hora_arribo: integer (nullable = true)
 |-- minuto_arribo: integer (nullable = true)
 |-- segundo_arribo: integer (nullable = true)


 Ricardo Madrigal Urencio 

+----

In [10]:
#Estadisticos Descriptivos
print(nombre)

ecobici.describe().show()


 Ricardo Madrigal Urencio 

+-------+--------------+------------------+------------------+---------------------+--------------+-----------------+------------------+----------+--------------------+------------------+------------------+------------------+---------------------+--------------+------------------+------------------+----------+-----------+------------------+------------------+------------------+
|summary|Genero_Usuario|      Edad_Usuario|              Bici|Ciclo_Estacion_Retiro|dia_sem_retiro|   num_dia_retiro|    num_mes_retiro|mes_retiro|         anio_retiro|       hora_retiro|     minuto_retiro|    segundo_retiro|Ciclo_Estacion_Arribo|dia_sem_arribo|    num_dia_arribo|    num_mes_arribo|mes_arribo|anio_arribo|       hora_arribo|     minuto_arribo|    segundo_arribo|
+-------+--------------+------------------+------------------+---------------------+--------------+-----------------+------------------+----------+--------------------+------------------+------------------+---

In [11]:
#Estadisticos Descriptivos de algunas columnas del DataFrame
columnas = ['Edad_Usuario', 'Bici', 'dia_sem_retiro', 'mes_retiro', 'dia_sem_arribo', 'mes_arribo']
ecobici.select(columnas).describe().show()

print(nombre)

+-------+------------------+------------------+--------------+----------+--------------+----------+
|summary|      Edad_Usuario|              Bici|dia_sem_retiro|mes_retiro|dia_sem_arribo|mes_arribo|
+-------+------------------+------------------+--------------+----------+--------------+----------+
|  count|           7718156|           7718075|       7718156|   7718156|       7718155|   7718155|
|   mean| 35.15434748921893| 9071.506474736252|          NULL|      NULL|          NULL|      NULL|
| stddev|10.355505009818907|2872.8982650072044|          NULL|      NULL|          NULL|      NULL|
|    min|              16.0|                78|        Friday|     April|        Friday|     April|
|    max|             119.0|             15339|     Wednesday| September|     Wednesday| September|
+-------+------------------+------------------+--------------+----------+--------------+----------+


 Ricardo Madrigal Urencio 



In [12]:
#Guardar el nuevo dataframe
ecobici.write.mode("overwrite").option("header", "true").csv("hdfs://namenode:9000/tmp/amd/ecobicilimpio")
print("Done")

print(nombre)

Done

 Ricardo Madrigal Urencio 

